# 中文篇章级句间关系分类 - LoRA微调实验(NPU版本)

## 实验说明

本实验使用 **MindSpore + mindnlp 0.5.1 + LoRA** 在 **DeepSeek-R1-Distill-Qwen-1.5B** 模型上进行微调。

### 任务目标
- **输入**：一个句子或对话片段
- **输出**：该句子所属的PDTB篇章关系分类（扩展/因果/比较/并列/其他）以及分类原因

### 版本信息
- mindnlp: 0.5.1
- mindspore: 2.7.0
- transformers: 4.57.1
- 数据类型: bfloat16

### 训练环境
- 镜像：mindspore_2_7-vllm-mindspeed-cann8_2alpha2_ubuntu22
- 实例规格：Ascend: 1*ascend-snt9b1|ARM: 24核 192GB

## 1. 导入必要的库

In [1]:
# 核心框架
import mindnlp
import mindspore

# 设置NPU上下文
mindspore.set_context(mode=mindspore.PYNATIVE_MODE, device_target="Ascend", device_id=0)

# 数据处理
from datasets import Dataset
import pandas as pd

# 模型和训练相关
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

# LoRA相关
from peft import LoraConfig, TaskType, get_peft_model

# 查看版本信息
print(f"mindnlp版本: {mindnlp.__version__}")
print(f"mindspore版本: {mindspore.__version__}")

/home/ma-user/work/lcl/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/work/lcl/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/work/lcl/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/work/lcl/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/work/lcl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found.

mindnlp版本: 0.5.0rc2
mindspore版本: 2.7.0


## 2. 加载数据集

In [2]:
# 数据路径
train_path = "/home/ma-user/work/data/train.json"
val_path = "/home/ma-user/work/data/val.json"

# 读取数据
df_train = pd.read_json(train_path)
df_val = pd.read_json(val_path)

# 转换为Dataset格式
ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)

# 查看数据集信息
print(f"训练集样本数: {len(ds_train)}")
print(f"验证集样本数: {len(ds_val)}")
print("\n数据集前3个样本:")
ds_train[:3]

训练集样本数: 9198
验证集样本数: 1500

数据集前3个样本:


{'content': ['他的有没有什么不足之处？我觉得他可以就是加一些他自己的感受，因为他如果光只说那些一系列的动作，就感觉很空白，没有什么情感在里面。',
  '星汉是什么？银河。',
  '对于花来说没有人欣赏是多么的悲惨，就像我们姑娘把自己打扮得花枝招展，却没有人欣赏一样是一种不幸'],
 'summary': ['扩展\n原因：前半句话提出问题，询问他的不足之处，后半句话则具体回答了我认为的他的不足之处，所以属于扩展关系。',
  '扩展\n原因：',
  '扩展\n原因：']}

## 3. 加载Tokenizer

In [ ]:
# 加载tokenizer
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    use_fast=False, 
    trust_remote_code=True
)

# 查看tokenizer信息
print(f"词汇表大小: {tokenizer.vocab_size}")
print(f"最大长度: {tokenizer.model_max_length}")
print(f"PAD token: {tokenizer.pad_token}")
print(f"EOS token: {tokenizer.eos_token}")
tokenizer

词汇表大小: 151643
最大长度: 16384
PAD token: <｜end▁of▁sentence｜>
EOS token: <｜end▁of▁sentence｜>


LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False

## 4. 数据预处理

In [ ]:
# 最大序列长度
MAX_LENGTH = 1024

def process_func(example):
    # 构建指令部分
    instruction = tokenizer(
        f"<|im_start|>system\n你是一位PDTB中文文本关系分析助手<|im_end|>\n"
        f"<|im_start|>user\n{example.get('content', '')}<|im_end|>\n"
        f"<|im_start|>assistant\n",
        add_special_tokens=False
    )

    # 构建回答部分
    response = tokenizer(
        f"{example.get('summary', '')}", 
        add_special_tokens=False
    )

    # 拼接 input_ids 和 attention_mask
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]

    # 构建 labels：指令部分设为 -100（不计算损失），只对回答部分计算损失
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

    # 截断到最大长度
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids, 
        "attention_mask": attention_mask, 
        "labels": labels
    }

# 处理训练集和验证集
print("开始处理训练集...")
tokenized_train = ds_train.map(process_func, remove_columns=ds_train.column_names)
print("训练集处理完成！")

print("\n开始处理验证集...")
tokenized_val = ds_val.map(process_func, remove_columns=ds_val.column_names)
print("验证集处理完成！")

# 查看处理后的第一个样本
print("\n处理后的第一个样本解码结果:")
print(tokenizer.decode(tokenized_train[0]['input_ids']))

开始处理训练集...


Map: 100%|██████████| 9198/9198 [00:07<00:00, 1174.64 examples/s]


训练集处理完成！

开始处理验证集...


Map: 100%|██████████| 1500/1500 [00:01<00:00, 1348.14 examples/s]

验证集处理完成！

处理后的第一个样本解码结果:
<|im_start|>system
你是一位PDTB中文文本关系分析助手<|im_end|>
<|im_start|>user
他的有没有什么不足之处？我觉得他可以就是加一些他自己的感受，因为他如果光只说那些一系列的动作，就感觉很空白，没有什么情感在里面。<|im_end|>
<|im_start|>assistant
扩展
原因：前半句话提出问题，询问他的不足之处，后半句话则具体回答了我认为的他的不足之处，所以属于扩展关系。<｜end▁of▁sentence｜>


## 5. 加载基础模型

In [ ]:
# 加载基础模型
print("正在加载模型，请稍候...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    ms_dtype=mindspore.bfloat16,
    device_map=0
)

# 显式将模型移动到NPU
model = model.to('npu:0')

# 开启梯度计算
model.enable_input_require_grads()

print("模型加载完成！")
print(f"模型参数量: {model.num_parameters():,}")

正在加载模型，请稍候...


`torch_dtype` is deprecated! Use `dtype` instead!


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
模型加载完成！
模型参数量: 1,777,088,000


## 6. 配置LoRA

In [6]:
# 配置LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 因果语言模型
    target_modules=[        # 要应用LoRA的模块（注意力层和FFN层）
        "q_proj", "k_proj", "v_proj", "o_proj",   # 注意力层
        "gate_proj", "up_proj", "down_proj"       # FFN层
    ],
    r=16,                   # LoRA秩
    lora_alpha=32,          # LoRA缩放因子
    lora_dropout=0.05,      # Dropout率
    inference_mode=False    # 训练模式
)

# 应用LoRA到模型
model = get_peft_model(model, lora_config)

# 打印可训练参数信息
model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,795,552,768 || trainable%: 1.0284


## 7. 配置训练参数

In [ ]:
# 定义训练参数
args = TrainingArguments(
    output_dir="./output",                    # 输出目录
    per_device_train_batch_size=4,            # batch size
    gradient_accumulation_steps=5,            # 梯度累积步数
    logging_steps=10,                         # 日志记录间隔
    num_train_epochs=3,                       # 训练轮数
    save_steps=100,                           # checkpoint保存间隔
    learning_rate=3e-5,                       # 学习率
    save_on_each_node=True,                   # 在每个节点上保存
)

print("训练参数配置完成！")
print(f"有效batch size: {args.p er_device_train_batch_size * args.gradient_accumulation_steps}")
print(f"总训练步数: {len(tokenized_train) // (args.per_device_train_batch_size * args.gradient_accumulation_steps) * args.num_train_epochs}")

训练参数配置完成！
有效batch size: 20
总训练步数: 1377


## 8. 创建Trainer并开始训练

In [8]:
# 创建Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

print("Trainer创建成功！")

Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
The model is already on multiple devices. Skipping the move to device specified in `args`.


Trainer创建成功！


In [9]:
# 开始训练
print("========== 开始训练 ==========")
print("="*50)
trainer.train()
print("\n========== 训练完成 ==========")

========== 开始训练 ==========


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss
10,4.547900
20,3.537700
30,2.483600
40,1.875100
50,1.731100
60,1.573200
70,1.487300
80,1.352500
90,1.325900
100,1.277600



========== 训练完成 ==========


## 9. 训练结果查看

In [10]:
# 查看训练结果文件
import os
print("训练输出目录内容:")
if os.path.exists('./output'):
    for item in sorted(os.listdir('./output')):
        print(f"  - {item}")
else:
    print("输出目录尚未创建")

训练输出目录内容:
  - checkpoint-100
  - checkpoint-1000
  - checkpoint-1100
  - checkpoint-1200
  - checkpoint-1300
  - checkpoint-1380
  - checkpoint-200
  - checkpoint-300
  - checkpoint-400
  - checkpoint-500
  - checkpoint-600
  - checkpoint-700
  - checkpoint-800
  - checkpoint-900
